In [125]:
import pandas as pd
import os
import google.generativeai as genai
import json
from pathlib import Path
import re

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""
from openai import OpenAI

client = OpenAI()


### Get Prompt

In [119]:
def get_prompt(DATAFRAME_SAMPLE_JSON):
    return """You are helping author STATIC benchmark tasks for a schema-alignment evaluation.

The goal is to transform ONE input dataframe into multiple fragmented, CSV-like tables
that simulate exports from different organizational systems.

The solver (purple agent) will be given ONLY the fragmented tables and must infer:
1) primary keys
2) join relationships
3) naming inconsistencies
4) a merged canonical schema

CRITICAL RULES (MUST FOLLOW EXACTLY):

A) VALUE PRESERVATION (NON-NEGOTIABLE)
- You are given ONE input dataframe.
- You MUST NOT change any cell values.
- You may ONLY:
  1) split the dataframe into multiple tables (column subsets)
  2) optionally subset rows (preserve exact values)
  3) reorder rows or columns
  4) rename columns
  5) rename tables
  6) drop columns
- You MUST NOT:
  - edit numbers, strings, dates, or IDs
  - normalize, clean, impute, or synthesize values
  - invent new identifiers or relationships

B) ROW TRACEABILITY
- Every output table MUST include a shared row identifier column copied from the input
  (e.g., "__row_id__" or an existing unique ID).
- Do NOT rename this row identifier.

C) TABLE NAMING (COLUMN-DRIVEN, REQUIRED)
- Table names MUST be inferred ONLY from their column content.
- Use short, lowercase, plural nouns.
- Examples:
  - (customer_id, customer_name, email) → "customers"
  - (order_id, order_date, amount) → "orders"
- Do NOT use generic or theme-based names.

D) DIFFICULTY DEFINITIONS
You MUST generate THREE tasks with increasing difficulty:

1) Easy task:
   - exactly 2 tables
   - obvious primary keys
   - exactly ONE simple naming inconsistency
     (e.g., case or underscore difference)

2) Medium task:
   - exactly 3 tables
   - mixed naming conventions
     (snake_case, camelCase, UPPER_CASE)
   - at least 2 join relationships

3) Hard task:
   - exactly 5 tables
   - complex relationship graph
   - ambiguous column names (e.g., "id", "department")
   - at least one of:
     * self-referential join
     * multi-hop join chain (A → B → C)

E) JOIN REALISM
- If ground_truth includes a join ["A.colX","B.colY"],
  then the values in those columns MUST overlap in the sample rows shown.

F) OUTPUT FORMAT
- Output ONLY valid JSON.
- No markdown, no comments, no explanations.

---

## OUTPUT FORMAT (MUST MATCH EXACTLY)

Return a JSON ARRAY of SIX objects, in this order:
1) easy task #1
2) easy task #2
3) medium task #1
4) medium task #2
5) hard task #1
6) hard task #2
Each object MUST follow this schema:

{
  "task_id": "string",
  "difficulty": "easy | medium | hard",
  "source": {
    "input_name": "string",
    "row_id_col": "string"
  },
  "tables": [
    {
      "name": "string",
      "columns": ["string", "..."],
      "sample_data": [
        {"column": "value", "...": "..."},
        {"column": "value", "...": "..."}
      ]
    }
  ],
  "provenance": {
    "table_name": {
      "output_column": "input_column"
    }
  },
  "ground_truth": {
    "primary_keys": {"table": "column"},
    "join_columns": [["table.col","table.col"]],
    "inconsistencies": ["string"],
    "merged_schema": {
      "unified_table_name": ["canonical_col", "..."]
    }
  }
}

NOTES:
- sample_data rows must include ALL listed columns.
- primary key columns must be unique in the sample rows.
- merged_schema column names should be canonicalized (snake_case preferred).
- provenance must cover EVERY output column in EVERY table.

---

## INPUT DATAFRAME (DO NOT CHANGE VALUES)"""+f"""

{DATAFRAME_SAMPLE_JSON}

---
""" + """

## FEW-SHOT EXAMPLE (EASY ONLY)

{
  "task_id": "easy_customers_orders",
  "difficulty": "easy",
  "source": {
    "input_name": "input.csv",
    "row_id_col": "__row_id__"
  },
  "tables": [
    {
      "name": "customers",
      "columns": ["__row_id__", "cust_id", "customer_name", "email"],
      "sample_data": [
        {"__row_id__": 0, "cust_id": 1, "customer_name": "Alice", "email": "alice@example.com"},
        {"__row_id__": 1, "cust_id": 2, "customer_name": "Bob", "email": "bob@example.com"}
      ]
    },
    {
      "name": "orders",
      "columns": ["__row_id__", "order_id", "customer_ID", "amount"],
      "sample_data": [
        {"__row_id__": 0, "order_id": 101, "customer_ID": 1, "amount": 99.99},
        {"__row_id__": 1, "order_id": 102, "customer_ID": 2, "amount": 149.50}
      ]
    }
  ],
  "provenance": {
    "customers": {
      "__row_id__": "__row_id__",
      "cust_id": "customer_id",
      "customer_name": "customer_name",
      "email": "email"
    },
    "orders": {
      "__row_id__": "__row_id__",
      "order_id": "order_id",
      "customer_ID": "customer_id",
      "amount": "amount"
    }
  },
  "ground_truth": {
    "primary_keys": {
      "customers": "cust_id",
      "orders": "order_id"
    },
    "join_columns": [
      ["customers.cust_id", "orders.customer_ID"]
    ],
    "inconsistencies": [
      "cust_id vs customer_ID (case and naming)"
    ],
    "merged_schema": {
      "customer_orders": [
        "customer_id",
        "customer_name",
        "email",
        "order_id",
        "amount"
      ]
    }
  }
}

---

## YOUR TASK

Using the SAME input dataframe:
- Generate SIX tasks:
  1) two EASY
  2) two MEDIUM
  3) two HARD
- Tasks of the same difficulty MUST differ in table splits, naming, or joins.
- Follow ALL rules above.
- Return ONLY a JSON ARRAY of six task objects.
"""

### Run Single Inference

In [120]:
def get_single_response_gpt(prompt):

    response = client.chat.completions.create(
        model="gpt-4.1",   # fast + cheap + good with tables
        messages=[
            {"role": "system", "content": "You are a careful data reasoning assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
    )

    return response.choices[0].message.content.strip()

In [121]:
def get_single_response(DATAFRAME_SAMPLE_JSON):
    prompt = get_prompt(DATAFRAME_SAMPLE_JSON)
    result = get_single_response_gpt(prompt)

    return result, prompt

### Run On One Dataset

In [143]:
df = pd.read_excel("CRM-Data-Analysis-With-Python.xlsx")
output_dir = "tasks"

In [122]:
curr_df = df[:5]

curr_df  = curr_df[curr_df.columns[:10]]

json_data = json.dumps(
    {"name": "source", "columns": list(curr_df.columns), "sample_data": curr_df.to_dict(orient="records")},
)
result, prompt = get_single_response(json_data)
parsed_results = json.loads(result)

In [138]:
def find_max_task_number(root_dir, difficulty):
    pattern = re.compile(rf"^task_{re.escape(difficulty)}_(\d+)$")
    max_number = -1

    for p in Path(root_dir).iterdir():
        if p.is_dir():
            match = pattern.match(p.name)
            if match:
                num = int(match.group(1))
                max_number = num if max_number is None else max(max_number, num)

    return max_number

In [141]:
def save_one_task(task_dir, task_dict):
    difficulty = task_dict['difficulty']
    folder_name = f"task_{difficulty}_{find_max_task_number(task_dir, difficulty)+1}"

    save_dir = os.path.join(task_dir, folder_name)
    os.makedirs(save_dir, exist_ok=False)

    # save all the tables
    for table in task_dict['tables']:
       data_dict = {x:[] for x in table['columns']} 

       for row in table['sample_data']:
           for key, val in row.items():
               data_dict[key].append(val)
       
       data_df = pd.DataFrame(data_dict)

       data_df.to_csv(os.path.join(save_dir, table['name']+".csv"), index=False, encoding="utf-8")

    # save the ground truth as json file
    with open(os.path.join(save_dir, "ground_truth.json"), "w", encoding="utf-8") as f:
        json.dump(task_dict['ground_truth'], f, ensure_ascii=False, indent=2)

In [144]:
for task in parsed_results:
    save_one_task(output_dir, task)